In [1]:
import pandas as pd

# Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.





In [2]:
df_rating = pd.read_csv('ml-latest-small/ratings.csv')
df_rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [4]:

df_rating_100 = df_rating.copy()
df_rating_100['count'] = 1
df_rating_100 = df_rating_100.groupby(['userId']).sum().reset_index()[['userId', 'count']].sort_values('userId', ascending=False).reset_index(drop=True)
df_rating_100.query('count >= 100', inplace = True)
df_rating_100 = df_rating_100.reset_index(drop = True)
df_rating_100.head()

,userId,count
0,671,115
1,665,434
2,664,519
3,659,142
4,656,128


In [5]:
df_first_rating = df_rating.copy()
df_first_rating = df_first_rating.groupby('userId')['timestamp'].agg(first_rating = 'min').reset_index()
df_first_rating.head()


,userId,first_rating
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044


In [6]:
df_finish_rating = df_rating.copy()
df_finish_rating = df_finish_rating.groupby('userId')['timestamp'].agg(finish_rating = 'max').reset_index()
df_finish_rating.head()


,userId,finish_rating
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [7]:
df_rating_100 = df_rating_100.merge(df_first_rating, on = ['userId'], how = 'left')
df_rating_100 = df_rating_100.merge(df_finish_rating, on = ['userId'], how = 'left')
df_rating_100['ltv'] = df_rating_100['finish_rating'] - df_rating_100['first_rating']
df_rating_100.head()

,userId,count,first_rating,finish_rating,ltv
0,671,115,1063500751,1074784735,11283984
1,665,434,992836298,1046967769,54131471
2,664,519,1343731283,1441911722,98180439
3,659,142,834598040,866207451,31609411
4,656,128,986240991,986244044,3053


In [8]:
x = df_rating_100['ltv'].mean()

In [9]:
print(f'среднее время жизни пользователей, поставивших более 100 оценок = {x:.0f}')

среднее время жизни пользователей, поставивших более 100 оценок = 39331749


# Задание  2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.




In [10]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [11]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [12]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [13]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [14]:
df_non_adress = client_base.merge(auto, on = ['client_id'], how = 'left')
df_non_adress = df_non_adress.merge(rzd, on = ['client_id'], how = 'left')
df_non_adress = df_non_adress.merge(air, on = ['client_id'], how = 'left')
df_non_adress = df_non_adress.fillna(0)
df_non_adress.head(8)

,client_id,address,auto_revenue,rzd_revenue,air_revenue
0,111,Комсомольская 4,0.0,1093.0,0.0
1,112,Энтузиастов 8а,0.0,2810.0,0.0
2,113,Левобережная 1а,57483.0,10283.0,0.0
3,114,Мира 14,83.0,5774.0,0.0
4,115,ЗЖБИиДК 1,912.0,981.0,81.0
5,116,Строителей 18,4834.0,0.0,4.0
6,117,Панфиловская 33,98.0,0.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


In [15]:
df_adress = client_base[['client_id']]
df_adress = df_adress.merge(auto, on = ['client_id'], how = 'left')
df_adress = df_adress.merge(rzd, on = ['client_id'], how = 'left')
df_adress = df_adress.merge(air, on = ['client_id'], how = 'left')
df_adress = df_adress.fillna(0)
df_adress.head(8)

,client_id,auto_revenue,rzd_revenue,air_revenue
0,111,0.0,1093.0,0.0
1,112,0.0,2810.0,0.0
2,113,57483.0,10283.0,0.0
3,114,83.0,5774.0,0.0
4,115,912.0,981.0,81.0
5,116,4834.0,0.0,4.0
6,117,98.0,0.0,13.0
7,118,0.0,0.0,173.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
